# Visualization
[Link to tutorial](https://octopus-code.org/documentation/13/tutorial/basics/visualization/)

In this tutorial we will explain how to visualize outputs from **Octopus**. Several different file formats are available in **Octopus** that are suitable for a variety of visualization software. We will not cover all of them here. See [Visualization](https://www.octopus-code.org/documentation//13/manual/visualization) for more information.


In [ ]:
!mkdir -p 4-visualisation

In [ ]:
cd 4-visualisation

## Benzene molecule

As an example, we will use the benzene molecule. At this point, the input should be quite familiar to you:


In [ ]:
%%writefile inp

stdout = "stdout_gs.txt"
stderr = "stderr_gs.txt"

CalculationMode = gs
UnitsOutput = eV_Angstrom

Radius = 5*angstrom
Spacing = 0.15*angstrom

%Output
  wfs
  density
  elf
  potential
%

OutputFormat = cube + xcrysden + dx + axis_x + plane_z

XYZCoordinates = "benzene.xyz"

Coordinates are in this case given in the `benzene.xyz` file. This file should look like:


In [ ]:
%%writefile benzene.xyz


12
   Geometry of benzene (in Angstrom)
   C  0.000  1.396  0.000
   C  1.209  0.698  0.000
   C  1.209 -0.698  0.000
   C  0.000 -1.396  0.000
   C -1.209 -0.698  0.000
   C -1.209  0.698  0.000
   H  0.000  2.479  0.000
   H  2.147  1.240  0.000
   H  2.147 -1.240  0.000
   H  0.000 -2.479  0.000
   H -2.147 -1.240  0.000
   H -2.147  1.240  0.000

Here we have used two new input variables:

* [Output](https://www.octopus-code.org/documentation//13/variables/output/output): this tells **Octopus** what quantities to output.
* [OutputFormat](https://www.octopus-code.org/documentation//13/variables/output/outputformat): specifies the format of the output files.

In this case we ask **Octopus** to output the wavefunctions (`wfs`), the density, the electron localization function (`elf`) and the Kohn-Sham potential. We ask **Octopus** to generate this output in several formats, that are required for the different visualization tools that we mention below. If you want to save some disk space you can just keep the option that corresponds to the program you will use. Take a look at the documentation on variables [Output](https://www.octopus-code.org/documentation//13/variables/output/output) and [OutputFormat](https://www.octopus-code.org/documentation//13/variables/output/outputformat) for the full list of possible quantities to output and formats to visualize them in.


In [ ]:
!octopus

In [ ]:
!ls static

## Visualization
We use postopus to collect and visualize the data

In [ ]:
import holoviews as hv
import matplotlib.pyplot as plt
import numpy as np
from holoviews import opts  # For setting defaults

from postopus import Run

# hv.extension('matplotlib')  # Uncomment to use matplotlib as backend
hv.extension("bokeh")  # Allow for interactive plots

# Choose color scheme similar to matplotlib
opts.defaults(
    opts.Image(cmap="viridis", width=500, height=400),
)

In [ ]:
# Create a Run object from the output files
run = Run(".")

In [ ]:
# Plot of energy at each interaction showing its convergence
fig = run.default.scf.convergence()["energy"].plot(
    title="Energy convergence",
    marker=".",
    markersize=10,
)
fig.set(xlabel="Iteration", ylabel="Energy (eV)");

Plot a slice of the density in the xy plane (z=0)

In [ ]:
def plot(s1):
    fig, ax = plt.subplots()
    s1.plot(ax=ax)
    ax.set_title("Density of benzene in the xy plane (z=0)")
    ax.set_xlabel("x (Å)")
    ax.set_ylabel("y (Å)")
    return fig, ax

In [ ]:
density = run.default.scf.density
xa = density(source="cube")[-1]  # Get the converged density
s1 = xa.sel(z=0, method="nearest")  # Select the xy plane (nearest data point to z=0)
fig, ax = plot(s1)  # Plot the density

Plot of 6 2D slice of the density along the x axis

In [ ]:
fig, axs = plt.subplots(
    3, 2, figsize=(16, 18)
)  # create a subplot with 2 rows, 3 columns
x_positions = np.linspace(-7.5, 5, 6)  # x positions from -7.5 to 5 in 6 steps
for ax, x in zip(axs.flat, x_positions):
    xa.sel(x=x, method="nearest").plot(
        ax=ax, x="y"
    )  # plot the slice nearest to x position

fig.tight_layout()  # avoid overlap of labels

Plot of the electron density in xy plane with an interactive slider to select the z position

In [ ]:
# Holoviews
hv.Dataset(xa).to(hv.Image, kdims=["x", "y"], dynamic=True).opts(
    colorbar=True, clim=(0, xa.values.max())
)

[Go to *5-Centering_Example.ipynb*](5-Centering_Example.ipynb)